In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.raster import ImageryLayer

gis = GIS()
gis.users.me

In [2]:
esri_imagery = ImageryLayer("https://civsci.esrigc.com/image/rest/services/heatmax_median_multivariate_annual_cw/ImageServer", gis=gis)
egis_imagery = ImageryLayer("https://gis.earthdata.nasa.gov/maphost/rest/services/EIC/heatmax_median_multivariate_annual/ImageServer", gis=gis)

In [3]:
# Set the parameters for the benchmark

GEOMETRY = {"spatialReference":{"wkid":4326},"x":-103.3665440972982,"y":0.995054807838955}
GEOMETRY_TYPE = "point"
MOSAIC_RULE = {"ascending":True,"multidimensionalDefinition":[{"variableName":"heatmax_ssp126","dimensionName":"StdTime","values":[[-628560000000,249868800000]],"isSlice":False},{"variableName":"heatmax_ssp245","dimensionName":"StdTime","values":[[-628560000000,249868800000]],"isSlice":False}]}
RETURN_FIRST_VALUE = False
INTERPOLATION = "RSP_NearestNeighbor"

In [4]:
def benchmark_imagery_layer(imagery_layer, parameters):
    return imagery_layer.get_samples(geometry=parameters[0], geometry_type=parameters[1], mosaic_rule=parameters[2], return_first_value_only=parameters[3], interpolation=parameters[4])

In [5]:
def benchmark_report(benchmark_output):
    print(
f'''
Out of {benchmark_output.repeat} runs, {benchmark_output.loops} loops each...

 Average   Best   Worst   stdev
|-------|-------|-------|-------|
| {round(benchmark_output.average, 3)} | {round(benchmark_output.best, 3)} | {round(benchmark_output.worst, 3)} | {round(benchmark_output.stdev, 3)} | seconds
''')
    return 

In [ ]:
esri_output = %timeit -o -r 10 -n 10 benchmark_imagery_layer(esri_imagery, [GEOMETRY, GEOMETRY_TYPE, MOSAIC_RULE, RETURN_FIRST_VALUE, INTERPOLATION])
benchmark_report(esri_output)

In [ ]:
egis_output = %timeit -o -r 10 -n 10 benchmark_imagery_layer(egis_imagery, [GEOMETRY, GEOMETRY_TYPE, MOSAIC_RULE, RETURN_FIRST_VALUE, INTERPOLATION])
benchmark_report(egis_output)

In [12]:
import logging
import time
import requests

# Set up logging
logging.basicConfig(filename='benchmark_log.log', level=logging.INFO, format='%(asctime)s - %(message)s')

def intercept_http_request(url, method='GET', **kwargs):
    start_time = time.time()
    
    if method.upper() == 'GET':
        response = requests.get(url, **kwargs)
    elif method.upper() == 'POST':
        response = requests.post(url, **kwargs)
    else:
        raise ValueError("Unsupported HTTP method. Use 'GET' or 'POST'.")
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # Log the request and response details
    logging.info(f"Method: {method}")
    logging.info(f"URL: {url}")
    logging.info(f"Elapsed Time: {elapsed_time} seconds")
    logging.info(f"Status Code: {response.status_code}")
    logging.info(f"Request Headers: {response.request.headers}")
    logging.info(f"Response Headers: {response.headers}")
    logging.info(f"Response Content: {response.text[:500]}")  # Log first 500 characters of the response content
    
    # Print the same information to the console
    print(f"Method: {method}")
    print(f"URL: {url}")
    print(f"Elapsed Time: {elapsed_time} seconds")
    print(f"Status Code: {response.status_code}")
    print(f"Request Headers: {response.request.headers}")
    print(f"Response Headers: {response.headers}")
    print(f"Response Content: {response.text[:500]}")  # Print first 500 characters of the response content
    
    return response, elapsed_time

In [13]:
def benchmark_imagery_layer(imagery_layer, parameters, method='GET'):
    url = imagery_layer.url  # Assuming imagery_layer has a 'url' attribute
    
    # Intercept the HTTP request and measure the time
    response, elapsed_time = intercept_http_request(url, method, params={
        'geometry': parameters[0],
        'geometryType': parameters[1],
        'mosaicRule': parameters[2],
        'returnFirstValueOnly': parameters[3],
        'interpolation': parameters[4]
    })
    
    return response

In [10]:
# Example parameters
esri_imagery_layer = ImageryLayer(url="https://civsci.esrigc.com/image/rest/services/heatmax_median_multivariate_annual_cw/ImageServer")
parameters = [GEOMETRY, GEOMETRY_TYPE, MOSAIC_RULE, RETURN_FIRST_VALUE, INTERPOLATION]

# Benchmark the imagery layer
response = benchmark_imagery_layer(esri_imagery_layer, parameters)

In [11]:
# Example parameters
egis_imagery_layer = ImageryLayer(url="https://gis.earthdata.nasa.gov/maphost/rest/services/EIC/heatmax_median_multivariate_annual/ImageServer")
parameters = [GEOMETRY, GEOMETRY_TYPE, MOSAIC_RULE, RETURN_FIRST_VALUE, INTERPOLATION]

# Benchmark the imagery layer
response = benchmark_imagery_layer(egis_imagery_layer, parameters)

In [ ]:
# Example parameters
esri_imagery_layer = ImageryLayer(url="https://civsci.esrigc.com/image/rest/services/heatmax_median_multivariate_annual_cw/ImageServer")
parameters = [GEOMETRY, GEOMETRY_TYPE, MOSAIC_RULE, RETURN_FIRST_VALUE, INTERPOLATION]

# Benchmark the imagery layer with GET request
response_get = benchmark_imagery_layer(esri_imagery_layer, parameters, method='GET')

# Benchmark the imagery layer with POST request
response_post = benchmark_imagery_layer(esri_imagery_layer, parameters, method='POST')

In [ ]:
# Example parameters
egis_imagery_layer = ImageryLayer(url="https://gis.earthdata.nasa.gov/maphost/rest/services/EIC/heatmax_median_multivariate_annual/ImageServer")
parameters = [GEOMETRY, GEOMETRY_TYPE, MOSAIC_RULE, RETURN_FIRST_VALUE, INTERPOLATION]

# Benchmark the imagery layer with GET request
response_get = benchmark_imagery_layer(egis_imagery_layer, parameters, method='GET')

# Benchmark the imagery layer with POST request
response_post = benchmark_imagery_layer(egis_imagery_layer, parameters, method='POST')